In [5]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random

In [6]:
kafka_topic_name = "Topic2"
kafka_bootstrap_servers = 'localhost:9092'

spark = SparkSession \
        .builder \
        .appName("Structured Streaming ") \
        .master("local[*]") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [7]:
df_transaction = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
        .option("subscribe", kafka_topic_name) \
        .option("startingOffsets", "earliest") \
        .load()

In [8]:
df1_transaction = df_transaction.selectExpr("CAST(value AS STRING)", "timestamp")


df_schema_string = "id INT,Account_No STRING,TRANSACTION_DETAILS STRING,Cheq_No STRING,WITHDRAWL_AMOUNT STRING,DEPOSIT_AMT STRING,BALANCE_AMT STRING"



df2_transaction= df1_transaction \
        .select(from_csv(col("value"), df_schema_string) \
                .alias("transaction"), "timestamp")


df3_transaction = df2_transaction.select("transaction.*", "timestamp")

    
df3_transaction.createOrReplaceTempView("transaction");
final_transaction = spark.sql("SELECT * FROM transaction")
Transaction_agg_write_stream = final_transaction \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("Transaction_Table") \
        .start()
Transaction_agg_write_stream.awaitTermination(1)

False

In [9]:
df = spark.sql("SELECT * FROM Transaction_Table")
df.show()

+---+-------------+-------------------+--------------------+----------------+-------------------+-----------+--------------------+
| id|   Account_No|TRANSACTION_DETAILS|             Cheq_No|WITHDRAWL_AMOUNT|        DEPOSIT_AMT|BALANCE_AMT|           timestamp|
+---+-------------+-------------------+--------------------+----------------+-------------------+-----------+--------------------+
|  0|409000611074'|2017-06-29 00:00:00|TRF FROM  Indiafo...|            NULL|2017-06-29 00:00:00|       0.00|2024-05-13 13:05:...|
|  1|409000611074'|2017-07-05 00:00:00|TRF FROM  Indiafo...|            NULL|2017-07-05 00:00:00|       0.00|2024-05-13 13:05:...|
|  2|409000611074'|2017-07-18 00:00:00|FDRL/INTERNAL FUN...|            NULL|2017-07-18 00:00:00|       0.00|2024-05-13 13:05:...|
|  3|409000611074'|2017-08-01 00:00:00|TRF FRM  Indiafor...|            NULL|2017-08-01 00:00:00|       0.00|2024-05-13 13:05:...|
|  4|409000611074'|2017-08-16 00:00:00|FDRL/INTERNAL FUN...|            NULL|2017-0

In [10]:
df.count()

100